In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [19]:
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

In [ ]:
file_name = "EXPORT_DATA_envoi_01_01_2022_30_04_2025.csv"   
df = pd.read_csv(file_name, 
                 nrows=1000000,
                 engine='python',  # Key change!
                 sep=";",
                on_bad_lines="skip",
                dtype=str,
                )

In [ ]:
 
df.head()


,RECPTCL_FID,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal
0,NaN,CC021659473ES,Insérer envoi dans sac (Srt),2024-01-31 20:11:00.000,ESPAGNE,8,NaN
1,NaN,CC021662747ES,Réception d'envoi du client (Srt),2024-01-31 12:31:00.000,ESPAGNE,1,NaN
2,NaN,CC021662747ES,Insérer envoi dans sac (Srt),2024-02-05 12:46:00.000,ESPAGNE,8,NaN
3,NaN,CC021663328ES,Réception d'envoi du client (Srt),2024-01-31 13:51:00.000,ESPAGNE,1,NaN
4,NaN,CC021663328ES,Insérer envoi dans sac (Srt),2024-02-01 12:32:00.000,ESPAGNE,8,NaN


In [23]:
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

file_name = "EXPORT_DATA_envoi_01_01_2022_30_04_2025.csv"   
df = pd.read_csv(file_name, 
                 nrows=1000000,
                 engine='python',  # Key change!
                 sep=";",
                on_bad_lines="skip",
                dtype=str,
                )

df["date"] = pd.to_datetime(df["date"])


fid = df["MAILITM_FID"].astype("string")

two_letter_suffix = fid.str[-2:].str.isalpha()

bad_suffix = df[~two_letter_suffix & fid.notna()]

suffix_ok = (
    df["MAILITM_FID"]
      .astype("string")
      .str[-2:]
      .str.isalpha()
      .fillna(False)          # treat missing IDs as bad
)

df = df[suffix_ok]

df['country'] = df["MAILITM_FID"].str[-2:]
df.head()

ID_COL = "MAILITM_FID"
df = df.sort_values([ID_COL, "date"], ascending=[True, True]).reset_index(drop=True)

cols_ignore = ["RECPTCL_FID", "country"]
cols_check  = [c for c in df.columns if c not in cols_ignore]

dup_mask = df.duplicated(subset=cols_check, keep=False)

num_duplicates = dup_mask.sum()

df = df.drop_duplicates(subset=cols_check, keep="first").reset_index(drop=True)

df = df.drop(columns=["RECPTCL_FID"])

df = df.sort_values(by=["MAILITM_FID", "date"])

# Calculate time to next step
df["duration_to_next_step"] = df.groupby("MAILITM_FID")["date"].shift(-1) - df["date"]

# Calculate total duration per ID
first_date = df.groupby("MAILITM_FID")["date"].transform("first")
last_date = df.groupby("MAILITM_FID")["date"].transform("last")
df["total_duration"] = last_date - first_date




In [ ]:
import pandas as pd

# Load main data and sorting centers list  # Replace as needed
sorting_centers = pd.read_csv("sorting_centers.csv")["établissement_postal"].tolist()

# Filter to only sorting centers
df = df[df["établissement_postal"].isin(sorting_centers)].copy()

# Make sure the data is sorted by package and time
df = df.sort_values(["MAILITM_FID", "date"])

# Assign visit_id for each separate visit (same package, same center, consecutive)
def get_visit_id(subdf):
    return (subdf["établissement_postal"] != subdf["établissement_postal"].shift()).cumsum()

df["visit_id"] = df.groupby("MAILITM_FID", group_keys=False).apply(get_visit_id)

# Compute duration per visit
visit_durations = (
    df.groupby(["établissement_postal", "MAILITM_FID", "visit_id"])["duration_to_next_step"]
    .sum()
    .reset_index()
)

# For each sorting center, aggregate stats including max duration and the package ID
def get_mailitm_fid_with_max_duration(subdf):
    idx = subdf["duration_to_next_step"].idxmax()
    return subdf.loc[idx, "MAILITM_FID"]

result = (
    visit_durations.groupby("établissement_postal")
    .agg(
        average_duration_per_visit=("duration_to_next_step", "mean"),
        number_of_visits=("duration_to_next_step", "count"),
        total_duration=("duration_to_next_step", "sum"),
        max_duration_per_visit=("duration_to_next_step", "max"),
    )
    .reset_index()
)

# Add the package ID with the max duration per center
result["mailitm_fid_with_max_duration"] = result["établissement_postal"].map(
    lambda center: get_mailitm_fid_with_max_duration(
        visit_durations[visit_durations["établissement_postal"] == center]
    )
)

# Save the result
result.to_csv("sorting_centers_average_visit_duration.csv", index=False)
print(result.head(10))
print("✅ Done! Results saved to sorting_centers_average_visit_duration.csv")


C:\Users\oussa\AppData\Local\Temp\ipykernel_26012\634767279.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["visit_id"] = df.groupby("MAILITM_FID", group_keys=False).apply(get_visit_id)


  établissement_postal average_duration_per_visit  number_of_visits  \
0  ALGER COLIS POSTAUX  3 days 23:44:18.106924653             12051   
1      ANNABA EL MARSA  3 days 17:58:23.579836435              1773   
2       CDD 20 AOUT 56     2 days 22:48:18.965750                 8   
3            CDD ADRAR  4 days 18:53:50.192636363                77   
4            CDD AFLOU     0 days 10:13:33.761700                10   
5        CDD AIN DEFLA  2 days 22:19:47.919698113               212   
6       CDD AIN MILILA  0 days 07:31:40.710564516                62   
7        CDD AIN SEFRA  0 days 08:42:56.422647058                17   
8   CDD AIN TEMOUCHENT  1 days 12:53:08.633261682               107   
9      CDD AIN-OUSSERA     1 days 21:22:44.844250                20   

              total_duration   max_duration_per_visit  \
0 48072 days 15:00:46.549000 377 days 02:29:57.326000   
1  6646 days 18:30:47.050000 315 days 23:46:45.770000   
2    23 days 14:26:31.726000  11 days 05:07:57.

In [5]:
df.head(20)

,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal,country,duration_to_next_step,total_duration
0,CA000020800LY,Insérer envoi dans sac (Srt),2022-02-03 10:19:00,NaN,8,NaN,LY,6 days 22:15:00,6 days 22:15:00
1,CA000020800LY,Insérer envoi dans sac (Srt),2022-02-10 08:34:00,"LIBYENNE, JAMAHIRIYA ARABE",8,NaN,LY,NaT,6 days 22:15:00
2,CA000020844RU,Réception d'envoi du client (Srt),2024-12-10 16:40:00,"RUSSIE, FÉDÉRATION DE",1,NaN,RU,NaT,0 days 00:00:00
3,CA000086085US,Réception d'envoi du client (Srt),2024-11-16 00:31:00,ÉTATS-UNIS,1,NaN,US,8 days 08:07:00,8 days 08:07:00
4,CA000086085US,Insérer envoi dans sac (Srt),2024-11-24 08:38:00,ÉTATS-UNIS,8,NaN,US,NaT,8 days 08:07:00
5,CA000094303US,Réception d'envoi du client (Srt),2024-11-18 11:00:00,ÉTATS-UNIS,1,NaN,US,3 days 05:53:00,3 days 05:53:00
6,CA000094303US,Insérer envoi dans sac (Srt),2024-11-21 16:53:00,ÉTATS-UNIS,8,NaN,US,NaT,3 days 05:53:00
7,CA000102985US,Réception d'envoi du client (Srt),2024-11-21 14:39:00,ÉTATS-UNIS,1,NaN,US,1 days 00:21:00,1 days 00:21:00
8,CA000102985US,Insérer envoi dans sac (Srt),2024-11-22 15:00:00,ÉTATS-UNIS,8,NaN,US,NaT,1 days 00:21:00
9,CA000132868US,Réception d'envoi du client (Srt),2024-11-27 13:56:00,ÉTATS-UNIS,1,NaN,US,2 days 11:32:00,2 days 11:32:00


In [ ]:
import pandas as pd
from pathlib import Path
import gc

# ── CONFIGURE THESE ──────────────────────────────────────────────
FILES = [
    "df_count_1.csv", 
    "df_count_2_part1.csv", 
    "df_count_2_part2.csv", 
    "df_count_3_part1.csv", 
    "df_count_3_part2.csv", 
    "df_count_4.csv"
]
PACKAGE_COL = "MAILITM_FID"   # <--- CHANGE THIS to your package ID column
POSTAL_COL  = "établissement_postal"
TARGET_POSTS = {"ALGER COLIS POSTAUX"}
# ─────────────────────────────────────────────────────────────────

all_packages = set()
target_packages = set()

for src in FILES:
    print(f"→ reading {src}")
    df = pd.read_csv(src, usecols=[PACKAGE_COL, POSTAL_COL])
    
    all_packages.update(df[PACKAGE_COL].unique())
    
    filtered = df[df[POSTAL_COL].isin(TARGET_POSTS)]
    target_packages.update(filtered[PACKAGE_COL].unique())
    
    del df, filtered
    gc.collect()

total_packages = len(all_packages)
total_target   = len(target_packages)
rate = total_target / total_packages if total_packages > 0 else 0

print(f"\nTotal unique packages: {total_packages}")
print(f'Packages with at least one row at {TARGET_POSTS}: {total_target}')
print(f"Rate: {rate:.2%}")

# Optionally, save IDs:
# pd.Series(list(target_packages)).to_csv("alger_related_packages.csv", index=False)


→ reading df_count_1.csv
→ reading df_count_2_part1.csv
→ reading df_count_2_part2.csv
→ reading df_count_3_part1.csv
→ reading df_count_3_part2.csv
→ reading df_count_4.csv

Total unique packages: 4571582
Packages with at least one row at {'ALGER COLIS POSTAUX'}: 4149596
Rate: 90.77%


In [27]:
import pandas as pd
from pathlib import Path
import gc

# ── CONFIGURE THESE ──────────────────────────────────────────────
FILES = [
    "df_count_1.csv", 
    "df_count_2_part1.csv", 
    "df_count_2_part2.csv", 
    "df_count_3_part1.csv", 
    "df_count_3_part2.csv", 
    "df_count_4.csv"
]
PACKAGE_COL = "MAILITM_FID"   # <--- CHANGE THIS to your package ID column
POSTAL_COL  = "établissement_postal"
TARGET_POSTS = {"ORAN COLIS POSTAUX"}
# ─────────────────────────────────────────────────────────────────

all_packages = set()
target_packages = set()

for src in FILES:
    print(f"→ reading {src}")
    df = pd.read_csv(src, usecols=[PACKAGE_COL, POSTAL_COL])
    
    all_packages.update(df[PACKAGE_COL].unique())
    
    filtered = df[df[POSTAL_COL].isin(TARGET_POSTS)]
    target_packages.update(filtered[PACKAGE_COL].unique())
    
    del df, filtered
    gc.collect()

total_packages = len(all_packages)
total_target   = len(target_packages)
rate = total_target / total_packages if total_packages > 0 else 0

print(f"\nTotal unique packages: {total_packages}")
print(f'Packages with at least one row at {TARGET_POSTS}: {total_target}')
print(f"Rate: {rate:.2%}")

# Optionally, save IDs:
# pd.Series(list(target_packages)).to_csv("alger_related_packages.csv", index=False)


→ reading df_count_1.csv
→ reading df_count_2_part1.csv
→ reading df_count_2_part2.csv
→ reading df_count_3_part1.csv
→ reading df_count_3_part2.csv
→ reading df_count_4.csv

Total unique packages: 4571582
Packages with at least one row at {'ORAN COLIS POSTAUX'}: 537400
Rate: 11.76%


In [28]:
import pandas as pd
from pathlib import Path
import gc

# ── CONFIGURE THESE ──────────────────────────────────────────────
FILES = [
    "df_count_1.csv", 
    "df_count_2_part1.csv", 
    "df_count_2_part2.csv", 
    "df_count_3_part1.csv", 
    "df_count_3_part2.csv", 
    "df_count_4.csv"
]
PACKAGE_COL = "MAILITM_FID"   # <--- CHANGE THIS to your package ID column
POSTAL_COL  = "établissement_postal"
TARGET_POSTS = {"ALGER COLIS POSTAUX", "ORAN COLIS POSTAUX" , "CONSTANTINE COLIS POSTAUX"}
# ─────────────────────────────────────────────────────────────────

all_packages = set()
target_packages = set()

for src in FILES:
    print(f"→ reading {src}")
    df = pd.read_csv(src, usecols=[PACKAGE_COL, POSTAL_COL])
    
    all_packages.update(df[PACKAGE_COL].unique())
    
    filtered = df[df[POSTAL_COL].isin(TARGET_POSTS)]
    target_packages.update(filtered[PACKAGE_COL].unique())
    
    del df, filtered
    gc.collect()

total_packages = len(all_packages)
total_target   = len(target_packages)
rate = total_target / total_packages if total_packages > 0 else 0

print(f"\nTotal unique packages: {total_packages}")
print(f'Packages with at least one row at {TARGET_POSTS}: {total_target}')
print(f"Rate: {rate:.2%}")

# Optionally, save IDs:
# pd.Series(list(target_packages)).to_csv("alger_related_packages.csv", index=False)


→ reading df_count_1.csv
→ reading df_count_2_part1.csv
→ reading df_count_2_part2.csv
→ reading df_count_3_part1.csv
→ reading df_count_3_part2.csv
→ reading df_count_4.csv

Total unique packages: 4571582
Packages with at least one row at {'ORAN COLIS POSTAUX', 'CONSTANTINE COLIS POSTAUX', 'ALGER COLIS POSTAUX'}: 4153911
Rate: 90.86%


In [26]:
import pandas as pd
import gc

# ── CONFIGURE THESE ──────────────────────────────────────────────
FILES = [
    "df_count_1.csv", 
    "df_count_2_part1.csv", 
    "df_count_2_part2.csv", 
    "df_count_3_part1.csv", 
    "df_count_3_part2.csv", 
    "df_count_4.csv"
]
PACKAGE_COL = "MAILITM_FID"   # <--- CHANGE to your package ID column name
POSTAL_COL  = "établissement_postal"
POST_1 = "ALGER COLIS POSTAUX"
POST_2 = "ORAN COLIS POSTAUX"
# ─────────────────────────────────────────────────────────────────

all_packages = set()
post1_packages = set()
post2_packages = set()

for src in FILES:
    print(f"→ reading {src}")
    df = pd.read_csv(src, usecols=[PACKAGE_COL, POSTAL_COL])

    # All unique packages
    all_packages.update(df[PACKAGE_COL].unique())

    # Who passed through POST_1?
    post1_packages.update(df.loc[df[POSTAL_COL] == POST_1, PACKAGE_COL].unique())

    # Who passed through POST_2?
    post2_packages.update(df.loc[df[POSTAL_COL] == POST_2, PACKAGE_COL].unique())

    del df
    gc.collect()

# Packages that passed through BOTH
both_packages = post1_packages & post2_packages

total_packages = len(all_packages)
total_both     = len(both_packages)
rate = total_both / total_packages if total_packages > 0 else 0

print(f"\nTotal unique packages: {total_packages}")
print(f'Packages that passed through BOTH "{POST_1}" AND "{POST_2}": {total_both}')
print(f"Rate: {rate:.2%}")

# Optionally, save the IDs of these packages
# pd.Series(list(both_packages)).to_csv("alger_both_packages.csv", index=False)


→ reading df_count_1.csv
→ reading df_count_2_part1.csv
→ reading df_count_2_part2.csv
→ reading df_count_3_part1.csv
→ reading df_count_3_part2.csv
→ reading df_count_4.csv

Total unique packages: 4571582
Packages that passed through BOTH "ALGER COLIS POSTAUX" AND "ORAN COLIS POSTAUX": 536217
Rate: 11.73%


In [5]:
import pandas as pd
from pathlib import Path
import gc

# ── CONFIGURE THESE ──────────────────────────────────────────────
TEMPLATE     = "df_with_durations_0{}.csv"   # e.g. df_with_durations_01.csv
FILE_NUMBERS = range(1, 8)                   # 1 … 7
OUT_FILE     = Path("unique_events.csv")     # where to save unique events
COL_NM       = "EVENT_TYPE_NM"
COL_CD       = "EVENT_TYPE_CD"
# ─────────────────────────────────────────────────────────────────

# Collect all event rows in a list
all_events = []

for i in FILE_NUMBERS:
    src = TEMPLATE.format(i)
    print(f"→ reading {src}")

    df = pd.read_csv(src, usecols=[COL_NM, COL_CD])   # only need these two columns

    all_events.append(df)
    del df
    gc.collect()

# Combine all event rows
combined = pd.concat(all_events, ignore_index=True)

# Drop duplicates on both columns together
unique_events = combined.drop_duplicates(subset=[COL_NM, COL_CD])

# Sort by EVENT_TYPE_CD (as integer if it's not already)
# If EVENT_TYPE_CD is string, convert to int first
unique_events[COL_CD] = unique_events[COL_CD].astype(int)
unique_events = unique_events.sort_values(by=COL_CD)

# Reset index for clean output
unique_events = unique_events.reset_index(drop=True)

# Save to CSV
unique_events.to_csv(OUT_FILE, index=False)

print("✓ All unique events extracted and saved")
print(f"Result saved to: {OUT_FILE.resolve()}")
print(unique_events)  # Show the result in the console too


→ reading df_with_durations_01.csv
→ reading df_with_durations_02.csv
→ reading df_with_durations_03.csv
→ reading df_with_durations_04.csv
→ reading df_with_durations_05.csv
→ reading df_with_durations_06.csv
→ reading df_with_durations_07.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_24416\3339846076.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_events[COL_CD] = unique_events[COL_CD].astype(int)


✓ All unique events extracted and saved
Result saved to: C:\Users\oussa\Desktop\alg_poste_stage\unique_events.csv
                                        EVENT_TYPE_NM  EVENT_TYPE_CD
0                   Réception d'envoi du client (Srt)              1
1            Expédier envoi à adresse nationale (Srt)              2
2            Recevoir envoi au bureau d'échange (Srt)              3
3                    Expédier envoi à la douane (Srt)              4
4                        Recevoir envoi au lieu (Srt)              5
5   Enregistrer raison de rétention d'envoi par la...              6
6                   Renvoyer envoi de la douane (Srt)              7
7                        Insérer envoi dans sac (Srt)              8
8                        Supprimer envoi du sac (Srt)              9
9   Enregistrer détails d'envoi au bureau d'échang...             11
10         Expédition d'envoi à l'étranger (EDI-reçu)             12
11           Recevoir envoi au bureau d'échange (Ent)     

In [3]:
file_name = "EXPORT_DATA_envoi_01_01_2022_30_04_2025.csv"   
df1 = pd.read_csv(file_name, 
                 nrows=100000,
                 engine='python',  # Key change!
                 sep=";",
                on_bad_lines="skip",
                dtype=str,
                )

In [10]:
import pandas as pd
file_name = "df_count_3_part1.csv" 
features = ["MAILITM_FID" , "date" ,"établissement_postal", "EVENT_TYPE_NM" , "EVENT_TYPE_CD" , "next_établissement_postal"]

df1 = pd.read_csv(file_name, 
                 nrows=100000,
                 engine='python',  # Key change!
                on_bad_lines="skip",
                dtype=str,
                )
df1 = df1[features]
df1["RECPTCL_FID"] = pd.NA

In [11]:
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"{filename} saved successfully.")
save_to_csv(df1, "df_test.csv")

df_test.csv saved successfully.


In [19]:
import pandas as pd

IN_FILE = "unique_etablissements.csv"
OUT_FILE = "sorting_centers.csv"
COL = "établissement_postal"   # adjust if your column is named differently

special_names = [
    "ALGER COLIS POSTAUX",
    "CONSTANTINE COLIS POSTAUX",
    "ANNABA EL MARSA",
    "ORAN COLIS POSTAUX"
]

df = pd.read_csv(IN_FILE)
df[COL] = df[COL].astype(str).str.strip()

mask = (
    df[COL].str.startswith("CDD")
    | df[COL].str.startswith("CTR")
    | df[COL].isin(special_names)
)

sorting_centers_df = df[mask].copy()
sorting_centers_df.to_csv(OUT_FILE, index=False)

print(f"Sorting centers list saved to: {OUT_FILE}")
print(sorting_centers_df.head())
print(f"Total sorting centers: {len(sorting_centers_df)}")
sorting_centers_df.to_csv(OUT_FILE, index=False)


Sorting centers list saved to: sorting_centers.csv
     établissement_postal
399   ALGER COLIS POSTAUX
444       ANNABA EL MARSA
1146       CDD 20 AOUT 56
1147            CDD ADRAR
1148            CDD AFLOU
Total sorting centers: 85


In [16]:
import pandas as pd
from pathlib import Path
import gc

# CONFIGURE THESE
TEMPLATE     = "df_with_durations_0{}.csv"
FILE_NUMBERS = range(1, 8)
ETAB_COL     = "établissement_postal"    # <-- adjust to match your exact column name!
UNIQUE_FILE  = Path("unique_etablissements.csv")

UNIQUE_FILE.unlink(missing_ok=True)

unique_etabs = set()

for i in FILE_NUMBERS:
    src = TEMPLATE.format(i)
    print(f"→ reading {src}")

    df = pd.read_csv(src)

    # Add unique 'etablissement postale' from this file
    unique_etabs.update(df[ETAB_COL].dropna().unique())

    del df
    gc.collect()

print("✓ all files processed")

# Save to DataFrame/CSV
unique_df = pd.DataFrame({ETAB_COL: sorted(unique_etabs)})
unique_df.to_csv(UNIQUE_FILE, index=False)
print(f"Unique etablissements saved to: {UNIQUE_FILE.resolve()}")


→ reading df_with_durations_01.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_26012\2294128847.py:19: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(src)


→ reading df_with_durations_02.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_26012\2294128847.py:19: DtypeWarning: Columns (0,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(src)


→ reading df_with_durations_03.csv
→ reading df_with_durations_04.csv
→ reading df_with_durations_05.csv
→ reading df_with_durations_06.csv
→ reading df_with_durations_07.csv
✓ all files processed
Unique etablissements saved to: C:\Users\oussa\Desktop\alg_poste_stage\unique_etablissements.csv


In [1]:
import pandas as pd

# Replace with your actual filename
file_path = "code_etablissement.xlsx"

# Reads the first sheet by default
df = pd.read_excel(file_path)

# To see the first few rows
df.head()


,bp_num_ips,bp_num_tm,code_upw,upw,bp_nm,type_etablissement,postal_cd
0,3684.0,664,35,UPW BOUMERDES,BORDJ MENAEIL,ETAB,35001
1,354.0,665,35,UPW BOUMERDES,BOUDOUAOU,ETAB,35003
2,898.0,668,35,UPW BOUMERDES,ZEMMOURI,ETAB,35012
3,351.0,669,35,UPW BOUMERDES,THENIA,ETAB,35005
4,3698.0,670,35,UPW BOUMERDES,HEMMADI,ETAB,35015


In [1]:
import pandas as pd

# Replace with your actual filename
file_path = "delai_international.xlsx"

# Reads the first sheet by default
df = pd.read_excel(file_path)

# To see the first few rows
df.head()


,orig_wilaya_cd,dest_wilaya_cd,type_envoi,delai
0,16,1,C,9
1,16,1,U,9
2,16,2,C,7
3,16,2,U,6
4,16,3,C,9


In [2]:
df.to_csv("delai_international.csv", index=False)
print("Saved as delai_international.csv")

Saved as delai_international.csv


In [2]:
import pandas as pd

df = pd.read_csv("code_etablissement.csv")

# Preview to confirm
df.head()


,bp_num_ips,bp_num_tm,code_upw,upw,bp_nm,type_etablissement,postal_cd
0,3684.0,664,35,UPW BOUMERDES,BORDJ MENAEIL,ETAB,35001
1,354.0,665,35,UPW BOUMERDES,BOUDOUAOU,ETAB,35003
2,898.0,668,35,UPW BOUMERDES,ZEMMOURI,ETAB,35012
3,351.0,669,35,UPW BOUMERDES,THENIA,ETAB,35005
4,3698.0,670,35,UPW BOUMERDES,HEMMADI,ETAB,35015


In [5]:
unique_type = df['type_etablissement'].unique()
unique_upw = df['upw'].unique()
unique_code_upw = df['code_upw'].unique()

# Print the results


In [6]:
print("Unique type_etablissement:")
print(unique_type)



Unique type_etablissement:
['ETAB  ' 'CTR   ' 'CTNI  ' 'CDD   ' 'CPX   ' 'ARPL  ' 'HYBD  ' 'CENTER']


In [20]:
print("\nUnique code_upw:")
print(list(unique_code_upw))


Unique code_upw:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 76, 77]


In [12]:
print("\nUnique upw:")
print(unique_upw)


Unique upw:
['UPW BOUMERDES' 'UPW TIZI OUZOU' 'UPW MEDEA' 'UPW BOUIRA' 'UPW TIPAZA'
 'UPW BLIDA' 'UPW ALGER CENTRE' 'UPW ORAN' 'UPW BATNA' 'UPW BEJAIA'
 'UPW SETIF' 'UPW ANNABA' 'UPW SKIKDA' 'UPW SAIDA' 'UPW SIDI BEL ABBES'
 'UPW AIN TEMOUCHENT ' 'UPW MASCARA ' 'UPW TLEMCEN' 'UPW MOSTAGANEM'
 'UPW CONSTANTINE' 'UPW ALGER EST' 'UPW ALGER OUEST' 'UPW ADRAR'
 'UPW TIMIMOUN' 'UPW BORDJ-BADJI-MOKHTAR' 'UPW LAGHOUAT'
 'UPW OUM EL BOUAGHI' 'UPW BISKRA' 'UPW OULED-DJELLAL' 'UPW TAMANRASSET'
 'UPW IN-GUEZZAM' 'UPW IN-SALAH' 'UPW TEBESSA' 'UPW TIARET' 'UPW DJELFA'
 'UPW JIJEL' 'UPW GUELMA' 'UPW MSILA' 'UPW OUARGLA' 'UPW TOUGGOURT'
 'UPW EL BAYADH' 'UPW ILLIZI' 'UPW DJANET' 'UPW BORDJ BOU ARRERIDJ'
 'UPW EL OUED' 'UPW EL-MEGHAIER' 'UPW KHENCHELA' 'UPW MILA'
 'UPW AIN DEFLA' 'UPW NAAMA' 'UPW CHLEF' 'UPW BECHAR' 'UPW TINDOUF'
 'UPW TISSEMSILT' 'UPW GHARDAIA' 'UPW EL TARF' 'UPW RELIZANE'
 'UPW SOUK AHRAS' 'UPW BENI-ABBES' 'UPW EL-MENIAA' 'UPW ALGER CENTER']


In [23]:

# Get unique upw/code_upw pairs
upw_df = df[['upw', 'code_upw']].drop_duplicates()

# Sort by code_upw (ascending, numeric)
upw_df = upw_df.sort_values('code_upw').reset_index(drop=True)

# Save to CSV
upw_df.to_csv("upw.csv", index=False)
print("Saved unique upw and code_upw pairs to upw.csv (sorted by code_upw)")


Saved unique upw and code_upw pairs to upw.csv (sorted by code_upw)


In [24]:
upw_df.head()

,upw,code_upw
0,UPW ADRAR,1
1,UPW CHLEF,2
2,UPW LAGHOUAT,3
3,UPW OUM EL BOUAGHI,4
4,UPW BATNA,5


In [25]:
for etab in df['type_etablissement'].unique():
    # Select all rows for this etablissement type
    subset = df[df['type_etablissement'] == etab]
    count = len(subset)
    print(f"\nType: {repr(etab)} — Count: {count}")
    print(subset.sample(n=min(10, count))) 


Type: 'ETAB  ' — Count: 4421
      bp_num_ips  bp_num_tm  code_upw                  upw  \
4299      2093.0       4408        47         UPW GHARDAIA   
2566      5054.0       2677        77      UPW ALGER OUEST   
2308      5669.0       5248        26            UPW MEDEA   
769       2158.0       2241        46  UPW AIN TEMOUCHENT    
1585      3481.0       3999        39          UPW EL OUED   
2135         NaN       4928        16     UPW ALGER CENTRE   
695       4597.0       2042        20            UPW SAIDA   
178        359.0       1003        10           UPW BOUIRA   
4399       836.0       2370        13          UPW TLEMCEN   
2055      5282.0       4825        47         UPW GHARDAIA   

                                             bp_nm type_etablissement  \
4299                                       SOUAREG             ETAB     
2566  RESIDENCE-UNIVERSITAIRE-TALEB-ABDERRAHMANE 1             ETAB     
2308                         MEDEA CHIKER SAID ALI             ETAB 

In [2]:
import pandas as pd
import gc

# --- Config ---
FILES = [
    "df_count_1.csv", 
    "df_count_2_part1.csv", 
    "df_count_2_part2.csv", 
    "df_count_3_part1.csv", 
    "df_count_3_part2.csv", 
    "df_count_4.csv"
]
MAILITM_FID_COL = "MAILITM_FID"   # Change if the column is named differently!

# --- Product type code map ---
PRODUCT_TYPE_MAP = {
    "EMS":       ("EA", "EZ"),
    "Letter Post Tracked": ("LA", "LZ"),
    "M bags":    ("MA", "MZ"),
    "IBRS":      ("QA", "QM"),
    "Letter Post Registered": ("RA", "RZ"),
    "Letter Post (goods)": ("UA", "UZ"),
    "Letter Post Insured": ("VA", "VZ"),
    "Parcel Post": ("CA", "CZ"),
    "ECOMPRO Parcel": ("HA", "HZ"),
    # Add more if needed!
}

def get_product_type(mailitm_fid):
    if not isinstance(mailitm_fid, str) or len(mailitm_fid) < 2:
        return "UNKNOWN"
    indicator = mailitm_fid[:2].upper()
    for ptype, (start, end) in PRODUCT_TYPE_MAP.items():
        if start <= indicator <= end:
            return ptype
    return "Other/Unknown"

# --- Collect all unique MAILITM_FID ---
all_ids = set()
for src in FILES:
    print(f"→ reading {src}")
    df = pd.read_csv(src, usecols=[MAILITM_FID_COL])
    all_ids.update(df[MAILITM_FID_COL].dropna().unique())
    del df
    gc.collect()

print("✓ All files processed.")

# --- Classify by product type ---
product_type_counts = {}
for fid in all_ids:
    ptype = get_product_type(fid)
    product_type_counts[ptype] = product_type_counts.get(ptype, 0) + 1

# --- Results ---
total_unique = len(all_ids)
summary = pd.DataFrame(list(product_type_counts.items()), columns=["Product Type", "Count"])
summary = summary.sort_values("Count", ascending=False).reset_index(drop=True)

print(f"\n=== SUMMARY ===")
print(f"Total unique MAILITM_FID: {total_unique}\n")
print(summary)

# Optionally save to CSV
# summary.to_csv("mailitm_fid_producttype_summary.csv", index=False)


→ reading df_count_1.csv
→ reading df_count_2_part1.csv
→ reading df_count_2_part2.csv
→ reading df_count_3_part1.csv
→ reading df_count_3_part2.csv
→ reading df_count_4.csv
✓ All files processed.

=== SUMMARY ===
Total unique MAILITM_FID: 4571582

             Product Type    Count
0  Letter Post Registered  2977469
1     Letter Post (goods)  1414250
2             Parcel Post   129110
3     Letter Post Tracked    50538
4     Letter Post Insured      215


In [3]:
import pandas as pd

# Load your CSV
df = pd.read_csv("delai_international.csv")

# Show structure for confirmation
print("Columns:", df.columns.tolist())
print(df.head())

# Build a mapping: for all destination wilayas, get the max delai where origin is 16
ref_delai = (
    df[df["orig_wilaya_cd"] == 16]
    .groupby("dest_wilaya_cd")["delai"]
    .max()
    .to_dict()
)

print("Reference max delai (origin 16):")
for k, v in ref_delai.items():
    print(f"16 -> {k}: {v}")

# --- Example of how you'd use this ---
def get_reference_delai(orig, dest):
    """Returns the reference delai for any origin in {16, 76, 77} and a destination."""
    if orig in {16, 76, 77}:
        return ref_delai.get(dest, None)
    else:
        # You can build a similar table for non-16 origins if needed
        return None

# Example: what's the reference delai from 77 (Algiers Ouest) to 25?
print("Delai 77 -> 25 (using 16 -> 25):", get_reference_delai(77, 25))


Columns: ['orig_wilaya_cd', 'dest_wilaya_cd', 'type_envoi', 'delai']
   orig_wilaya_cd  dest_wilaya_cd type_envoi  delai
0              16               1          C      9
1              16               1          U      9
2              16               2          C      7
3              16               2          U      6
4              16               3          C      9
Reference max delai (origin 16):
16 -> 1: 9
16 -> 2: 7
16 -> 3: 9
16 -> 4: 8
16 -> 5: 7
16 -> 6: 8
16 -> 7: 8
16 -> 8: 9
16 -> 9: 7
16 -> 10: 7
16 -> 11: 9
16 -> 12: 8
16 -> 13: 7
16 -> 14: 7
16 -> 15: 7
16 -> 16: 6
16 -> 17: 7
16 -> 18: 7
16 -> 19: 7
16 -> 20: 7
16 -> 21: 8
16 -> 22: 7
16 -> 23: 7
16 -> 24: 8
16 -> 25: 7
16 -> 26: 7
16 -> 27: 7
16 -> 28: 7
16 -> 29: 7
16 -> 30: 9
16 -> 31: 7
16 -> 32: 9
16 -> 33: 9
16 -> 34: 7
16 -> 35: 7
16 -> 36: 8
16 -> 37: 9
16 -> 38: 7
16 -> 39: 12
16 -> 40: 8
16 -> 41: 7
16 -> 42: 7
16 -> 43: 7
16 -> 44: 7
16 -> 45: 8
16 -> 46: 7
16 -> 47: 7
16 -> 48: 7
16 -> 49: 9
16 -> 

In [7]:
import pandas as pd

# Load your table
df = pd.read_csv("delai_international.csv")

# Build lookup: (origin, dest, type_envoi) → delai
# (for 16 → X, 16 → 76, 16 → 77)
lookup = {}
for _, row in df.iterrows():
    lookup[(row["orig_wilaya_cd"], row["dest_wilaya_cd"], row["type_envoi"])] = row["delai"]

def get_delai(origin, dest, type_envoi):
    # If origin in (76, 77), treat as 16
    if origin in [76, 77]:
        origin = 16
    # If dest in (16, 76, 77) and origin not 16/76/77, treat as 16→origin (reverse)
    if dest in [16, 76, 77] and origin not in [16, 76, 77]:
        return lookup.get((16, origin, type_envoi))
    # Otherwise: normal lookup
    return lookup.get((origin, dest, type_envoi))

# === EXAMPLES ===
# 16 → 25: normal
print("16→25 U:", get_delai(16, 25, "U"))
# 76 → 25: treat as 16 → 25
print("76→25 U:", get_delai(76, 25, "U"))
# 16 → 76: as in table
print("16→76 U:", get_delai(16, 76, "U"))
# 25 → 16: treat as 16 → 25
print("25→16 U:", get_delai(25, 16, "U"))
# 25 → 77: treat as 16 → 25
print("25→77 U:", get_delai(25, 77, "U"))
# 25 → 19: normal lookup
print("25→19 U:", get_delai(25, 19, "U"))


16→25 U: 6
76→25 U: 6
16→76 U: 5
25→16 U: 6
25→77 U: 6
25→19 U: None
